# empymod - marine CSEM

In [1]:
import empymod
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
%matplotlib widget

In [4]:
off = np.linspace(500, 10000, 101)

In [5]:
resp = empymod.bipole(
    src=(0, 0, -250, 0, 0),  # x, y, z, azimuth, dip
    rec=(off, off*0, -300, 0, 0),
    depth=(0, -300, -2000, -2500),
    res=(2e14, 0.3, 1, 100, 1),
    freqtime=1,
    verb=1,
)

In [ ]:

fig1, ax9 = plt.subplots()
ax9.set_ylabel('Amplitude (V/m)')
ax9.set_xlabel('Offset (m)')
ax9.set_yscale('log')
    
    
    
def f(x):
    resp = empymod.bipole(
        src=(0, 0, -250, 0, 0),  # x, y, z, azimuth, dip
        rec=(off, off*0+x, -300, 0, 0),
        depth=(0, -300, -2000, -2500),
        res=(2e14, 0.3, 1, 100, 1),
        freqtime=1,
        verb=1,
    )

    
    ax9.plot(off, resp.amp())
    
interact(f, x=widgets.FloatSlider(min=0, max=1000, step=10, value=0));

interactive(children=(FloatSlider(value=0.0, description='x', max=1000.0, step=10.0), Output()), _dom_classes=…

In [ ]:
fig2, ax = plt.subplots()

ax.plot(off, resp.amp())
ax.set_ylabel('Amplitude (V/m)')
ax.set_xlabel('Offset (m)')
ax.set_yscale('log')

In [ ]:
model = {
    'src': (0, 0, -250, 0, 0),  # x, y, z, azimuth, dip
    'rec': (off, off*0, -300, 0, 0),
    'depth': (0, -300, -1000, -1500),
    'freqtime': 1,
    'verb': 1,
}

In [ ]:
resp_bg = empymod.bipole(res=(2e14, 0.3, 1, 1, 1), **model)
resp_tg = empymod.bipole(res=(2e14, 0.3, 1, 100, 1), **model)

In [ ]:
fig3, ax = plt.subplots()

ax.set_title(f"Electric field, $f=${model['freqtime']} Hz")
ax.plot(off, resp_tg.amp(), 'C3')
ax.plot(off, resp_bg.amp())
ax.set_ylabel('Amplitude (V/m)')
ax.set_xlabel('Offset (m)')
ax.set_yscale('log')

In [ ]:
fig4, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharex=True, constrained_layout=True)

fig4.suptitle(f"Electric field, $f=${model['freqtime']} Hz")

ax1.plot(off, resp_tg.amp(), 'C3')
ax1.plot(off, resp_bg.amp())
ax1.set_ylabel('Amplitude (V/m)')
ax1.set_xlabel('Offset (m)')
ax1.set_yscale('log')

ax2.set_title(f"Electric field, $f=${model['freqtime']} Hz")
ax2.plot(off, resp_tg.pha(deg=True), 'C3')
ax2.plot(off, resp_bg.pha(deg=True))
ax2.set_ylabel('Phase (°)')
ax2.set_xlabel('Offset (m)')

In [ ]:
empymod.Report()